In [1]:
import pandas as pd
import numpy
import pandas
import joblib
import operator
import re
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 


In [2]:

# Membaca data dari file CSV
data = pd.read_csv('jobs.csv')

In [3]:
# Melihat informasi dasar tentang data
print(data.info())

# Melihat statistik deskriptif tentang data
print(data.describe())

# Melihat beberapa baris pertama data
print(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_id          252 non-null    int64 
 1   status           252 non-null    object
 2   city             252 non-null    object
 3   job_title        252 non-null    object
 4   organization_id  252 non-null    int64 
 5   contracts        252 non-null    object
 6   description      252 non-null    object
 7   skills           252 non-null    object
dtypes: int64(2), object(6)
memory usage: 15.9+ KB
None
          user_id  organization_id
count  252.000000       252.000000
mean    88.980159        91.555556
std     60.288513        57.789707
min      2.000000         0.000000
25%     34.000000        39.000000
50%     81.500000        88.500000
75%    141.250000       140.250000
max    201.000000       199.000000
   user_id     status        city                       job_title  \
0     

In [4]:
# Menghapus kolom yang tidak diperlukan
data = data.drop(columns=['status','city','organization_id','description'])

# Menghapus baris dengan nilai yang hilang
data = data.dropna()

In [5]:
# Melihat informasi dasar tentang data
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    252 non-null    int64 
 1   job_title  252 non-null    object
 2   contracts  252 non-null    object
 3   skills     252 non-null    object
dtypes: int64(1), object(3)
memory usage: 8.0+ KB
None


In [6]:
# Melihat statistik deskriptif tentang data
print(data.describe())

          user_id
count  252.000000
mean    88.980159
std     60.288513
min      2.000000
25%     34.000000
50%     81.500000
75%    141.250000
max    201.000000


In [7]:
# Melihat beberapa baris pertama data
print(data.head())

   user_id                       job_title contracts  \
0       55            Medical Receptionist        AL   
1       90     Clinical Research Associate        AL   
2        2  Territory Sales Representative        AL   
3        3                      Bus Driver        AL   
4        4         Foreign Service Officer        AL   

                                              skills  
0  {'skills': {'training': 1, 'certifications': 1...  
1  {'skills': {'hardware': 2, 'communication': 1,...  
2  {'skills': {'communication': 4, 'e-commerce': 1}}  
3  {'skills': {'mobile': 1, 'communication': 2, '...  
4  {'skills': {'training': 4, 'designer': 1, 'pro...  


In [8]:
data.to_csv('data_cleaned.csv', index=False)

dataCleaned = pd.read_csv('data_cleaned.csv')

In [9]:
dataCleaned.nunique()

user_id      200
job_title    222
contracts      1
skills       151
dtype: int64

In [10]:
print(dataCleaned.head())

   user_id                       job_title contracts  \
0       55            Medical Receptionist        AL   
1       90     Clinical Research Associate        AL   
2        2  Territory Sales Representative        AL   
3        3                      Bus Driver        AL   
4        4         Foreign Service Officer        AL   

                                              skills  
0  {'skills': {'training': 1, 'certifications': 1...  
1  {'skills': {'hardware': 2, 'communication': 1,...  
2  {'skills': {'communication': 4, 'e-commerce': 1}}  
3  {'skills': {'mobile': 1, 'communication': 2, '...  
4  {'skills': {'training': 4, 'designer': 1, 'pro...  


In [11]:
print("Rows:",dataCleaned.shape[0])


print("Columns:",dataCleaned.shape[1])


print("List:",dataCleaned.columns.tolist())
     

Rows: 252
Columns: 4
List: ['user_id', 'job_title', 'contracts', 'skills']


In [12]:
jobs = dataCleaned['job_title'].unique()
jobs

array(['Medical Receptionist', 'Clinical Research Associate',
       'Territory Sales Representative', 'Bus Driver',
       'Foreign Service Officer', 'Sales Analyst', 'Strategy Consultant?',
       'Cost Engineer', 'Solutions Consultant - Electrical Engineering',
       'Power Regulator', 'Conservation Technician',
       'Manager of Engineering', 'Forecast Systems Manager',
       'Water Services Technician',
       'Senior Analyst Retail Analytics and Reporting',
       'Total Quality Management Director', 'HVAC Mechanic',
       'Cognos Business Development Architect', 'School Social Worker',
       'Treasurer', 'Manufacturing Customer Service Representative',
       'Quality Assurance Manager', 'Night Auditor',
       'Business Intelligence Developer', 'Solid Waste Field Technician',
       'Healthcare Consultant', 'Organic Lab Worker',
       'Compliance Engineer',
       'Chief of Career Pathways and Integrated Learning',
       'Assistant Coach', 'IT Support Staff', 'Meeting Sp

In [13]:
newData = dataCleaned[['user_id', 'job_title','skills']]
newData

,user_id,job_title,skills
0,55,Medical Receptionist,"{'skills': {'training': 1, 'certifications': 1..."
1,90,Clinical Research Associate,"{'skills': {'hardware': 2, 'communication': 1,..."
2,2,Territory Sales Representative,"{'skills': {'communication': 4, 'e-commerce': 1}}"
3,3,Bus Driver,"{'skills': {'mobile': 1, 'communication': 2, '..."
4,4,Foreign Service Officer,"{'skills': {'training': 4, 'designer': 1, 'pro..."
...,...,...,...
247,74,Assistant General Manager,"{'skills': {'http': 2, 'postfix': 2, 'mobile':..."
248,8,Operations Agent,"{'skills': {'communication': 1, 'reporting': 2}}"
249,13,Event Coordinator,"{'skills': {'reporting': 2, 'ai': 5, 'communic..."
250,17,Mutual Fund Analyst,"{'skills': {'mobile': 2, 'communication': 1, '..."


In [14]:
newData['skills']

0      {'skills': {'training': 1, 'certifications': 1...
1      {'skills': {'hardware': 2, 'communication': 1,...
2      {'skills': {'communication': 4, 'e-commerce': 1}}
3      {'skills': {'mobile': 1, 'communication': 2, '...
4      {'skills': {'training': 4, 'designer': 1, 'pro...
                             ...                        
247    {'skills': {'http': 2, 'postfix': 2, 'mobile':...
248     {'skills': {'communication': 1, 'reporting': 2}}
249    {'skills': {'reporting': 2, 'ai': 5, 'communic...
250    {'skills': {'mobile': 2, 'communication': 1, '...
251    {'skills': {'hardware': 3, 'mobile': 1, 'coach...
Name: skills, Length: 252, dtype: object

In [15]:
def delete_spec_chars(input): 
    #Hapus karakter unik

    regex = r'[^a-zA-Z0-9\s]'
    output = re.sub(regex,'',input)
    return output

#List job and skills
skills = []
new_skills=[]
for j in jobs:
   

 
    d=[]
    listinlist=[]
    for i, row in newData.iterrows():
        if(row['job_title']==j):
            d.append(row['skills'])
    print(d)

  
    #Hapus karakter unik
    doc = delete_spec_chars(str(d))

    #Hilangkan angka
    doc = re.sub(r'\d+','',doc)

    
    doc=re.sub(r'skills','',doc)

    #Applying word tokenization to extract tokens
    tokens = word_tokenize(doc) 
    print("document is: ")
    print(doc)
    print(" tokens are: ")

    tokens= [word.lower() for word in tokens]
    print(tokens)
    
    #To remove duplicate skills 
    tokens=list(set(tokens))

    #Adding skills for one job
    skills.append(tokens)

    listinlist.append(doc)
    new_skills.append(listinlist)

  
print(skills)

["{'skills': {'training': 1, 'certifications': 1, 'http': 1, 'processing': 1, 'implementation': 1}}", "{'skills': {'training': 1, 'certifications': 1, 'http': 1, 'processing': 1, 'implementation': 1, 'documentation': 2, 'programming': 1, 'prototyping': 1, 'tech support': 1, 'penetration testing': 1, 'data processing': 1, 'technical': 8, 'networks': 1}}"]
document is: 
 training  certifications  http  processing  implementation   training  certifications  http  processing  implementation  documentation  programming  prototyping  tech support  penetration testing  data processing  technical  networks 
 tokens are: 
['training', 'certifications', 'http', 'processing', 'implementation', 'training', 'certifications', 'http', 'processing', 'implementation', 'documentation', 'programming', 'prototyping', 'tech', 'support', 'penetration', 'testing', 'data', 'processing', 'technical', 'networks']
["{'skills': {'hardware': 2, 'communication': 1, 'ensemble': 1}}"]
document is: 
 hardware  communi

In [16]:
new_skills

[[' training  certifications  http  processing  implementation   training  certifications  http  processing  implementation  documentation  programming  prototyping  tech support  penetration testing  data processing  technical  networks '],
 [' hardware  communication  ensemble '],
 [' communication  ecommerce '],
 [' mobile  communication  design   ai  communication  mobile   ai  diagnostics '],
 [' training  designer  programming  implementation  communication  collaboration  processing  coding  hardware  telecom  coaching  html  design  troubleshooting  internet  technical  electronics  php  networks  web development  backend '],
 [' git  http  nodejs  mobile  communication  svg  nodejs  ecommerce  html  design  internet  angular  angular '],
 [' mobile  communication  collaboration  training  ecommerce   training  data collection  processing  implementation  communication  data mining  innovation  machine learning  mobile  engineering  design  deep learning  go  mathematics  techn

In [17]:
skills

[['support',
  'training',
  'documentation',
  'implementation',
  'testing',
  'programming',
  'technical',
  'networks',
  'tech',
  'processing',
  'prototyping',
  'http',
  'certifications',
  'data',
  'penetration'],
 ['communication', 'hardware', 'ensemble'],
 ['communication', 'ecommerce'],
 ['mobile', 'design', 'communication', 'ai', 'diagnostics'],
 ['programming',
  'technical',
  'communication',
  'electronics',
  'designer',
  'implementation',
  'html',
  'internet',
  'networks',
  'processing',
  'development',
  'collaboration',
  'design',
  'coding',
  'training',
  'backend',
  'coaching',
  'php',
  'telecom',
  'troubleshooting',
  'hardware',
  'web'],
 ['mobile',
  'html',
  'internet',
  'nodejs',
  'communication',
  'git',
  'angular',
  'http',
  'svg',
  'design',
  'ecommerce'],
 ['mobile',
  'technical',
  'learning',
  'communication',
  'innovation',
  'go',
  'mining',
  'deep',
  'engineering',
  'mathematics',
  'implementation',
  'machine',
  '

In [18]:
len(skills)

222

In [19]:
total_skills=[]
for val in skills:
    for keys in val:
        if keys not in total_skills:
            total_skills.append(keys)
        else:
            continue

total_skills=list(set(total_skills))
print(total_skills)

print("Total skills : ", len(total_skills))

['service', 'technical', 'foundation', 'website', 'critical', 'cloud', 'iptables', 'reporting', 'problem', 'patterns', 'designer', 'visio', 'databases', 'jquery', 'tech', 'framework', 'volleyball', 'processing', 'development', 'vbnet', 'data', 'media', 'neural', 'coaching', 'lte', 'git', 'postfix', 'ajax', 'http', 'wordpress', 'solution', 'troubleshooting', 'assembly', 'analytics', 'technologies', 'linux', 'spark', 'manager', 'presentation', 'operating', 'fortran', 'hadoop', 'analytical', 'iphone', 'certifications', 'mobile', 'swing', 'documentation', 'thinking', 'word', 'communication', 'innovation', 'go', 'insights', 'support', 'sas', 'python', 'prestashop', 'wimax', 'collaboration', 'design', 'extjs', 'software', 'penetration', 'tableau', 'shiny', 'windows', 'php', 'priorities', 'social', 'renewable', 'eclipse', 'multitasking', 'system', 'teamwork', 'pascal', 'customer', 'nodejs', 'analysis', 'big', 'javascript', 'mentoring', 'web', 'time', 'programming', 'modeling', 'deep', 'engine

In [20]:

column_headers=['Jobs']

for val in total_skills:
    column_headers.append(val)

print(column_headers)
print("total number of columns are: ", len(column_headers))

['Jobs', 'service', 'technical', 'foundation', 'website', 'critical', 'cloud', 'iptables', 'reporting', 'problem', 'patterns', 'designer', 'visio', 'databases', 'jquery', 'tech', 'framework', 'volleyball', 'processing', 'development', 'vbnet', 'data', 'media', 'neural', 'coaching', 'lte', 'git', 'postfix', 'ajax', 'http', 'wordpress', 'solution', 'troubleshooting', 'assembly', 'analytics', 'technologies', 'linux', 'spark', 'manager', 'presentation', 'operating', 'fortran', 'hadoop', 'analytical', 'iphone', 'certifications', 'mobile', 'swing', 'documentation', 'thinking', 'word', 'communication', 'innovation', 'go', 'insights', 'support', 'sas', 'python', 'prestashop', 'wimax', 'collaboration', 'design', 'extjs', 'software', 'penetration', 'tableau', 'shiny', 'windows', 'php', 'priorities', 'social', 'renewable', 'eclipse', 'multitasking', 'system', 'teamwork', 'pascal', 'customer', 'nodejs', 'analysis', 'big', 'javascript', 'mentoring', 'web', 'time', 'programming', 'modeling', 'deep',

In [21]:

#List dalam dataframe.
data_list = []

for index, obj in enumerate(skills):
    inner_list=[]
    # print(obj)

    for val in column_headers:
        if val=='Jobs':
            inner_list.append(jobs[index])
        else:
            if val in obj:
                inner_list.append(1)
            else:
                inner_list.append(0)

    data_list.append(inner_list)

In [22]:
secData = pd.DataFrame(data_list,columns=column_headers)
secData.to_csv('data_cleaned2.csv', index=False)
secData

,Jobs,service,technical,foundation,website,critical,cloud,iptables,reporting,problem,...,prototyping,tools,hive,management,assembleur,computing,network,organizational,attention,hardware
0,Medical Receptionist,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,Clinical Research Associate,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,Territory Sales Representative,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bus Driver,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Foreign Service Officer,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,Environmental Services Representative,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
218,Assistant General Manager,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
219,Operations Agent,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
220,Event Coordinator,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:

#Convert dataframe ke matrix
secData_matrix= numpy.array(secData)
secData_matrix


array([['Medical Receptionist', 0, 1, ..., 0, 0, 0],
       ['Clinical Research Associate', 0, 0, ..., 0, 0, 1],
       ['Territory Sales Representative', 0, 0, ..., 0, 0, 0],
       ...,
       ['Operations Agent', 0, 0, ..., 0, 0, 0],
       ['Event Coordinator', 0, 0, ..., 0, 0, 0],
       ['Mutual Fund Analyst', 0, 0, ..., 0, 0, 0]], dtype=object)

In [24]:

def union(job1,job2):
    result = list(set(job1) | set(job2))
    return result


In [25]:
def intersection(job1,job2):
    #Calculate intersection
    result = list(set(job1) & set(job2))
    return result

In [27]:
def calculate_similiarity():
    
    #Simpan similiarity each job
    dict_val={}
    for index, obj in enumerate(skills):
        inner_list=[]
        for next_index,next_obj in enumerate(skills):
            if(index==next_index):
                inner_list.append(1.0)
            
            else:
                union_result=union(obj, next_obj)
                intersection_result=intersection(obj, next_obj)

                inner_list.append(len(intersection_result)/len(union_result))

        dict_val[jobs[index]] = inner_list

     
    return dict_val
sim=calculate_similiarity()
print(sim)

{'Medical Receptionist': [1.0, 0.0, 0.0, 0.0, 0.1935483870967742, 0.04, 0.1724137931034483, 0.17647058823529413, 0.13043478260869565, 0.0, 0.0, 0.0, 0.05555555555555555, 0.0, 0.05263157894736842, 0.09090909090909091, 0.0, 0.0, 0.0, 0.13636363636363635, 0.09523809523809523, 0.0, 0.047619047619047616, 0.038461538461538464, 0.14285714285714285, 0.0, 0.047619047619047616, 0.05555555555555555, 0.0, 0.0625, 0.0, 0.14285714285714285, 0.0, 0.6, 0.041666666666666664, 0.0, 0.0, 0.04, 0.0, 0.0, 0.0, 0.0, 0.0625, 0.0, 0.0, 0.06896551724137931, 0.1111111111111111, 0.1, 0.12, 0.05263157894736842, 0.09523809523809523, 0.21052631578947367, 0.1111111111111111, 0.0, 0.11538461538461539, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14285714285714285, 0.0, 0.0, 0.0, 0.0, 0.09523809523809523, 0.11764705882352941, 0.0, 0.0, 0.0, 0.0, 0.0, 0.037037037037037035, 0.058823529411764705, 0.05263157894736842, 0.0, 0.0, 0.04, 0.2, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.11764705882352941, 0.0, 0.041666666666666664, 0.058823529411

In [28]:
def calulate_query_similarity(skills_query):

    dict={}

    for index,obj in enumerate(skills):
        union_result=union(skills_query, obj)
        intersection_result=intersection(skills_query, obj)

        dict[jobs[index]]=(len(intersection_result)/len(union_result))

    return dict


**Input Skill**

In [29]:
def recommend():

    skills_input=input("Please Enter Your Skills:  ")
    #Input word, word, word, ...

    #Hapus special character
    skills_preprocessed=delete_spec_chars(skills_input)

    #Tokenize skills
    tokens = word_tokenize(skills_preprocessed)

    #Converting skills to lower case
    tokens= [word.lower() for word in tokens]

    skills_query=[word for word in tokens if word not in stop_words and len(word) > 1] #Removing stopwords 
    
    print(skills_query)

    res=calulate_query_similarity(skills_query)


    #Find highest matched in descending order
    sorted_d = dict( sorted(res.items(), key=operator.itemgetter(1),reverse=True))
   
    return res,sorted_d

tanimoto_similarity_val,sorted_d=recommend()

print("\nBefore sorting\n")
print(tanimoto_similarity_val)

print("\nAfter sorting\n")
print(sorted_d)

['big']

Before sorting

{'Medical Receptionist': 0.0, 'Clinical Research Associate': 0.0, 'Territory Sales Representative': 0.0, 'Bus Driver': 0.0, 'Foreign Service Officer': 0.0, 'Sales Analyst': 0.0, 'Strategy Consultant?': 0.0, 'Cost Engineer': 0.0, 'Solutions Consultant - Electrical Engineering': 0.0, 'Power Regulator': 0.0, 'Conservation Technician': 0.0, 'Manager of Engineering': 0.0, 'Forecast Systems Manager': 0.0, 'Water Services Technician': 0.0, 'Senior Analyst Retail Analytics and Reporting': 0.0, 'Total Quality Management Director': 0.0, 'HVAC Mechanic': 0.0, 'Cognos Business Development Architect': 0.0, 'School Social Worker': 0.0, 'Treasurer': 0.0, 'Manufacturing Customer Service Representative': 0.0, 'Quality Assurance Manager': 0.0, 'Night Auditor': 0.0, 'Business Intelligence Developer': 0.0, 'Solid Waste Field Technician': 0.0, 'Healthcare Consultant': 0.0, 'Organic Lab Worker': 0.0, 'Compliance Engineer': 0.0, 'Chief of Career Pathways and Integrated Learning': 0.0

In [30]:
top_match=list(sorted_d.keys())[0]

print(sim[top_match])

[0.12, 0.06666666666666667, 0.07142857142857142, 0.125, 0.06060606060606061, 0.14285714285714285, 0.18518518518518517, 0.26666666666666666, 0.0, 0.14285714285714285, 0.0, 0.06666666666666667, 0.13333333333333333, 0.0, 0.2, 0.047619047619047616, 0.0, 0.07142857142857142, 0.0, 0.0, 0.0, 0.058823529411764705, 0.05263157894736842, 0.08695652173913043, 0.15384615384615385, 0.0, 0.0, 0.0625, 0.07142857142857142, 0.0, 0.07142857142857142, 0.0, 0.05555555555555555, 0.1875, 0.0, 0.1111111111111111, 0.07692307692307693, 0.14285714285714285, 0.07142857142857142, 0.043478260869565216, 0.0, 0.0625, 0.07142857142857142, 0.07692307692307693, 0.07692307692307693, 0.11538461538461539, 0.058823529411764705, 0.17647058823529413, 1.0, 0.0, 0.05, 0.0, 0.21739130434782608, 0.0, 0.125, 0.0, 0.07142857142857142, 0.0, 0.0, 0.0625, 0.0, 0.2, 0.07692307692307693, 0.07142857142857142, 0.0, 0.0625, 0.0, 0.0, 0.06666666666666667, 0.06666666666666667, 0.058823529411764705, 0.0, 0.0, 0.04, 0.0, 0.125, 0.0, 0.07142857

In [31]:
def calculate_top(new_sorted_d):
    count=0
    top5=[]
    for item in new_sorted_d.keys():
        #Finding top5 documents

        if (count != 5):
            top5.append(item)
        else: 
            break
        count=count+1
    
    return top5

     

new_dict={}
for index,obj in enumerate(sim[top_match]):
    new_dict[jobs[index]]=obj
    
print(new_dict)


new_sorted_d=dict( sorted(new_dict.items(), key=operator.itemgetter(1),reverse=True))

print("\nnew sorted dict is:")
print("\n\n", new_sorted_d)

{'Medical Receptionist': 0.12, 'Clinical Research Associate': 0.06666666666666667, 'Territory Sales Representative': 0.07142857142857142, 'Bus Driver': 0.125, 'Foreign Service Officer': 0.06060606060606061, 'Sales Analyst': 0.14285714285714285, 'Strategy Consultant?': 0.18518518518518517, 'Cost Engineer': 0.26666666666666666, 'Solutions Consultant - Electrical Engineering': 0.0, 'Power Regulator': 0.14285714285714285, 'Conservation Technician': 0.0, 'Manager of Engineering': 0.06666666666666667, 'Forecast Systems Manager': 0.13333333333333333, 'Water Services Technician': 0.0, 'Senior Analyst Retail Analytics and Reporting': 0.2, 'Total Quality Management Director': 0.047619047619047616, 'HVAC Mechanic': 0.0, 'Cognos Business Development Architect': 0.07142857142857142, 'School Social Worker': 0.0, 'Treasurer': 0.0, 'Manufacturing Customer Service Representative': 0.0, 'Quality Assurance Manager': 0.058823529411764705, 'Night Auditor': 0.05263157894736842, 'Business Intelligence Develo

**Hasil**

In [32]:
#To retrieve top 5 document:
top5=calculate_top(new_sorted_d)

print("Recommended 5 jobs:")
print(top5)

Recommended 5 jobs:
['Project Manager', 'Data Architect', 'Cost Engineer', 'Operations Research Analysis Manager', 'Senior Analyst Retail Analytics and Reporting']
